# ColabでGoogleDriveをマウントする方法を説明します。

## 背景
Colabでは、ランタイムを削除するとDisk上のファイルがクリアされます。RusyBotはシステム上にSqlite3のDBをつくっているためランタイム削除のたびにDBがクリアされることになります。

そこで、DBをGoogleDriveに置く方法を説明します。

この方法はローカルPCで余裕のあるディスクにDBを構築したい場合にも有効です。

In [ ]:
! pip install rbot

In [ ]:
from rbot import BinanceConfig
from rbot import BinanceConfig

In [ ]:
# Google Driveをマウントする
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
# DBの保存先をGoogle Driveに変更する
config = BinanceConfig.BTCUSDT
config.db_base_dir = '/content/drive'

In [ ]:
# Binanceのインスタンスを作成する
binance = Binance(config)

binance

In [ ]:
# データをダウンロードする
binance.download(ndays=1, verbose=True)